# APS 4

Funções padrões

In [90]:
from funcoesTermosol import plota, importa, geraSaida

In [91]:
import math
import numpy as np

A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.

In [92]:
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xls')

In [93]:
print([nn,nm,nc,nr])

[3, 3, 2, 3]


In [94]:
Inc

array([[1.0e+00, 2.0e+00, 2.1e+11, 2.0e-04],
       [2.0e+00, 3.0e+00, 2.1e+11, 2.0e-04],
       [3.0e+00, 1.0e+00, 2.1e+11, 2.0e-04]])

In [95]:
R

array([[0.],
       [2.],
       [3.]])

In [96]:
F

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 150.],
       [-100.]])

In [97]:
N

array([[0. , 0. , 0.3],
       [0. , 0.4, 0.4]])

In [98]:
N.shape

(2, 3)

### Função para calcular distâncias, cossenos e senos

In [99]:
def calc_dist_cs(N, Inc):
    l_list = []
    c_list = []
    s_list = []
    for i in range(N.shape[1]):
        if i == N.shape[1]-1:
            l = (math.sqrt((N[0][i] - N[0][0])**2 + (N[1][i] - N[1][0])**2))
            c = (N[0][0]-N[0][i])/l
            s = (N[1][0]-N[1][i])/l
            l_list.append(l)
            c_list.append(c)
            s_list.append(s)
        else:
            l = (math.sqrt((N[0][i] - N[0][i+1])**2 + (N[1][i] - N[1][i+1])**2))
            c = (N[0][i+1]-N[0][i])/l
            s = (N[1][i+1]-N[1][i])/l
            l_list.append(l)
            c_list.append(c)
            s_list.append(s)
    l_array = np.c_[l_list]
    c_array = np.c_[c_list]
    s_array = np.c_[s_list]
    Inc = np.hstack((Inc, l_array))
    Inc = np.hstack((Inc, c_array))
    Inc = np.hstack((Inc, s_array))
    return Inc

ex:

In [100]:
Inc = calc_dist_cs(N,Inc)
Inc

array([[ 1.0e+00,  2.0e+00,  2.1e+11,  2.0e-04,  4.0e-01,  0.0e+00,
         1.0e+00],
       [ 2.0e+00,  3.0e+00,  2.1e+11,  2.0e-04,  3.0e-01,  1.0e+00,
         0.0e+00],
       [ 3.0e+00,  1.0e+00,  2.1e+11,  2.0e-04,  5.0e-01, -6.0e-01,
        -8.0e-01]])

### Função para achar matrizes de rigidez

In [107]:
def matrizes_rigidez(Inc, nm):
    matrizes = np.zeros(shape=(nm,4,4))   # Considerando a treliça com duas extremidades
    i = 0
    for cs in Inc:
        rigidez = cs[2]*cs[3]/cs[4]  # E*A/L para todos os membros
        #print(rigidez)
        c = cs[5]
        s = cs[6]
        matriz_cs = [[c**2 ,     c*s,   -(c**2), -(c*s) ],
                     [c*s ,      s**2,  -(c*s),  -(s**2)],
                     [-(c**2),  -(c*s),  c**2,    c *s  ],
                     [-(c*s),   -(s**2), c*s,     s**2  ]]
        #print(matriz_cs)
        matriz_cs = np.multiply(matriz_cs, rigidez)
        matrizes[i,:,:] = np.add(matrizes[i,:,:], matriz_cs)

        i+=1
    return matrizes


ex

In [109]:
a= matrizes_rigidez(Inc,nm)
a

array([[[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00,  1.050e+08,  0.000e+00, -1.050e+08],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00, -1.050e+08,  0.000e+00,  1.050e+08]],

       [[ 1.400e+08,  0.000e+00, -1.400e+08,  0.000e+00],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [-1.400e+08,  0.000e+00,  1.400e+08,  0.000e+00],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00]],

       [[ 3.024e+07,  4.032e+07, -3.024e+07, -4.032e+07],
        [ 4.032e+07,  5.376e+07, -4.032e+07, -5.376e+07],
        [-3.024e+07, -4.032e+07,  3.024e+07,  4.032e+07],
        [-4.032e+07, -5.376e+07,  4.032e+07,  5.376e+07]]])

### Função para matriz de rigidez global

In [ ]:
def rigidez_global(Inc, matrizes_rigidez, nm):
    matriz_global = np.zeros(shape=(nn*2,nn*2))   # Considerando 2 graus de liberdade~
    

    


A funcao 'geraSaida' cria um arquivo nome.txt contendo as reacoes de apoio Ft, 
deslocamentos Ut, deformacoes Epsi, forcas Fi e tensoes Ti internas. 
As entradas devem ser vetores coluna.